In [1]:
import glob
import json
import wandb
import pandas as pd
from collections import defaultdict

from config.train_shakespeare_char import *

import sys

In [2]:
config = {'backend': 'nccl', 'device': 'cuda', 'dtype': 'bfloat16', 'compile': True}
wandb.init(project=wandb_project, name=wandb_run_name, config=config, sync_tensorboard=True)
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: m-motta. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# run experiments:

vary:

n_iters  - to check if profiling changes with more or less iterations --it should not

        for (n_heads*h_size=const) vary: 

            n_heads 
            
            h_size

            seq_len  (seq length affect number of operations for fix head size and nheads, as it increased the projections dimensions, i.e. sizes of K, V, Q)

In [5]:
def collect_system_metrics(username, project):

    api = wandb.Api()
    runs = api.runs(f"{username}/{project}")
    system_metrics = defaultdict(dict) 
    
    for run in runs:
        if run.state =='finished':            
            system_metrics[run.name][run.id] = run.history(stream='events') #run.history() is a pandas data frame 
    
    return system_metrics

sm = collect_system_metrics("m-motta" , 'profile-attention-nano-gpt')

In [ ]:
#'-n_head'+str(n_head)+'-h_size'+str(n_embd)+'-seq_len'+str(block_size)

About logged runtimes:

there is a difference between the runtime from .history() and .history(stream='events). Firstly, because events are restricted to the GPU, but probably also because the system is checked at specific intervals/checked once again after iterations are finished.

In [6]:
def compute_average_runtime(system_metrics):
    
    runtimes = {}
    for params in system_metrics.keys():
        print('params',params)
        runtimes[params] = 0
        count = 0
        for id in system_metrics[params].keys():
            print('id',id)
            print('.iloc[-1]',system_metrics[params][id]._runtime.iloc[-1])
            runtimes[params]+= system_metrics[params][id]._runtime.iloc[-1]
            count += 1
            print('')
        runtimes[params] = runtimes[params]/count
    
    return runtimes    

compute_average_runtime(sm)

params flash-n_iters400-n_head6-h_size384-seq_len256
id unt2z7wo
.iloc[-1] 65.89632

params slow-n_iters400-n_head6-h_size384-seq_len256
id rnnz8csp
.iloc[-1] 77.098952



{'flash-n_iters400-n_head6-h_size384-seq_len256': 65.89632,
 'slow-n_iters400-n_head6-h_size384-seq_len256': 77.098952}

In [65]:
def aggregate_system_metrics(username, project):

    #this is appending all the runs, I don't think I'll need this

    api = wandb.Api()
    runs = api.runs(f"{username}/{project}")
    system_metrics = {'flash-attention': [], 'slow-attention': []}
    for run in runs:
        system_metrics[run.name].append(run.history(stream='events')) #run.history() is a pandas data frame 

    for key, group in system_metrics.items():
        if group:
            system_metrics[key] = pd.concat(group, axis=0, join='outer', ignore_index=True)
    
    return system_metrics

#system_metrics = aggregate_system_metrics("m-motta" , 'profile-attention-nano-gpt')

In [69]:
def filter_metrics(system_metrics, list_of_matches):
    #selects specific metris (columns) from the .history() dataframe, based on string matches
    pattern = '|'.join(list_of_matches)

    def split_select(s):
        return s.split('system.')[1]

    system_metrics = {
        key: df.filter(regex=pattern, axis=1).rename(columns=split_select)
        for key, df in system_metrics.items()
    }

    return system_metrics

In [71]:
system_metrics = aggregate_system_metrics("m-motta" , 'profile-attention-nano-gpt')
system_metrics = filter_metrics(system_metrics, ['gpu', 'disk'])

In [73]:
system_metrics['flash-attention'].head()

,gpu.0.powerPercent,gpu.0.powerWatts,disk.out,disk.\.usageGB,gpu.0.temp,gpu.0.memory,gpu.0.gpu,gpu.0.memoryAllocatedBytes,disk.\.usagePercent,disk.in,gpu.0.memoryAllocated
0,9.94,8.02,3.82,174.91,47.60,13.93,7.93,8.574687e+08,20.3,0.00,9.99
1,12.16,10.33,10.62,174.91,47.93,3.73,11.93,8.721968e+08,20.3,0.00,10.16
2,8.33,6.29,16.45,174.91,47.20,6.87,16.60,8.805635e+08,20.3,0.00,10.26
3,12.51,10.52,24.49,174.91,47.47,7.87,14.40,8.823680e+08,20.3,0.00,10.28
4,11.70,9.69,36.57,174.89,47.40,6.27,17.00,9.035754e+08,20.3,0.01,10.52


In [62]:

system_metrics['flash-attention'].head()

,system.gpu.0.powerPercent,system.gpu.0.powerWatts,system.disk.out,system.disk.\.usageGB,system.gpu.0.temp,system.gpu.0.memory,system.gpu.0.gpu,system.gpu.0.memoryAllocatedBytes,system.disk.\.usagePercent,system.disk.in,system.gpu.0.memoryAllocated
0,9.94,8.02,3.82,174.91,47.60,13.93,7.93,8.574687e+08,20.3,0.00,9.99
1,12.16,10.33,10.62,174.91,47.93,3.73,11.93,8.721968e+08,20.3,0.00,10.16
2,8.33,6.29,16.45,174.91,47.20,6.87,16.60,8.805635e+08,20.3,0.00,10.26
3,12.51,10.52,24.49,174.91,47.47,7.87,14.40,8.823680e+08,20.3,0.00,10.28
4,11.70,9.69,36.57,174.89,47.40,6.27,17.00,9.035754e+08,20.3,0.01,10.52


In [48]:
params = ['gpu', 'disk']
metrics = system_metrics['flash-attention'].columns
metrics = [m.split('system.')[1] for m in metrics if any([p in m for p in params])]
metrics

['gpu.0.powerPercent',
 'gpu.0.powerWatts',
 'disk.out',
 'disk.\\.usageGB',
 'gpu.0.temp',
 'gpu.0.memory',
 'gpu.0.gpu',
 'gpu.0.memoryAllocatedBytes',
 'disk.\\.usagePercent',
 'disk.in',
 'gpu.0.memoryAllocated']

In [21]:
#['schemaVersion', 'deviceProperties', 'distributedInfo', 'with_flops', 'with_modules', 'with_stack', 'traceEvents', 'traceName']
#only deviceProperties and traceEvents contain relevant information
data = []
for run in glob.glob(f"{out_dir}/**/*.pt.trace.json", recursive=True):
  #  print('\n',run)
    with open(run) as jsonFile:
        data.append(json.load(jsonFile))
       
        


Notes from paper:

All models are trained with the same hyperparameters for 400K steps.

We run all implementations with mixed-precision training (PyTorch AMP).

Speedup also changes when we increase the head dimension. Each block
requires more memory, so we need to use smaller block sizes to fit into SRAM. Figure 6 shows speedup with
head dimension 128 on an A100 (batch size 16, 12 heads). We see less speedup overall—but we can still see
significant speedup (up to 3×) with a causal mask, where half the blocks are masked out.
                     
We confirm that the memory footprint
of FlashAttention scales linearly with seq. length and is up to 3× faster than standard attention for
common seq. lengths (up to 2K). We confirm that runtime of block-sparse FlashAttention scales linearly
in seq. length and is faster than all existing approximate attention baselines.
                     
We train the model on 8×A100-80GB GPUs. Each training run takes between 16 and 19 minutes, and we
average the results of 10 runs.
                     
attension head, seq length and block size